# import timesheets

In [1]:
import os
import json
import hashlib
import traceback
from dateutil import parser
from datetime import datetime
from util.google_sheet import GGSheet

In [2]:
from util.frappyclient import FrappeClient

In [23]:
# %env ACERP_ADMINISTRATOR=Administrator
# %env ACERP_ADMINISTRATOR_PASS=

In [4]:
conn = FrappeClient("https://acerp.pandion.vn")
conn.login(os.environ['ACERP_ADMINISTRATOR'], os.environ['ACERP_ADMINISTRATOR_PASS'])

{'message': 'Logged In', 'home_page': '/app', 'full_name': 'Administrator'}

In [5]:
from util.util import mapping_row_with_attr, mapping_row_with_time_log, is_row_empty

In [6]:
from util.ts import Ts

In [7]:
ts = Ts(conn)

In [8]:
row_of_date = 3
row_start   = 5
row_end     = 1200

In [9]:
# import_key for task: task_id||
def handle_timesheet_row_with_parent_task(idx, row_data, row_attrs, row_date_header, company):

    # print(f"  Handle parent task at row: {idx}\n"
    #       f"  import_key: {row_attrs['import_key']}\n"
    #       f"  project_code: {row_attrs['project_code']}\n"
    #       f"  phase_name: {row_attrs['phase_name']}\n"
    #       f"  subject: {row_attrs['subject']}\n"
    #       f"  task: {row_attrs['task']}")
    task_doc = ts.get_task_doc_by_fields({
        "subject": row_attrs['subject'], 
        "project_code": row_attrs['project_code'], 
        "phase_name": row_attrs['phase_name'], 
        "activity_code": row_attrs['activity_code'],
        "is_group": 1})

    if task_doc:
        return task_doc
    else:
        # create new
        payload = {
            "subject": row_attrs["subject"],
            "project_code": row_attrs["project_code"],
            "phase_name": row_attrs["phase_name"],
            "activity_code": row_attrs['activity_code'],
            "progress": row_attrs["progress"],
            "is_group": 1,
            "task_status": row_attrs["task_status"],
        }
        print(f"{payload}")
        return ts.create_task(payload)

In [10]:
# import_key for task: timesheet_id||
def handle_timesheet_row_as_task(idx, row_data, row_attrs, row_date_header, 
                                 parent_task_row, parent_task_doc, company):
    time_logs_date, time_logsdate_string = mapping_row_with_time_log(row_data, row_date_header)
    print(time_logs_date)
    print(time_logsdate_string)
    
    # print(f"  Handle task at row: {idx}\n"
    #       f"  import_key: {row_attrs['import_key']}\n"
    #       f"  project_code: {row_attrs['project_code']}\n"
    #       f"  phase_name: {row_attrs['phase_name']}\n"
    #       # f"  subject: {row_attrs['subject']}\n"
    #       f"  employee_name: {row_attrs['employee_name']}\n"
    #       f"  task: {row_attrs['task']}\n"
    #       f"  parent task row: {parent_task_row}\n"
    #       f"  parent task doc: {parent_task_doc}\n"
    #       f"  time_logs: {time_logs_date}")

    # filter by: 
        # employee_code, project_code, parent_task_doc, 
        # task_no (related position with parent_task), -> no need
        # task
        # first day time logs
    parent_related = idx - parent_task_row
    ts_timelog = time_logsdate_string[0] if len(time_logsdate_string) > 0 else ""
    
    ts_name = ts.get_timesheet_name_by_fields({
        "employee_code": row_attrs['employee_code'],
        "project_code": row_attrs['project_code'],
        "task": row_attrs['task'],
        "parent_task": parent_task_doc['subject'],
        # "parent_related": parent_related,
        "timelog": ts_timelog,
        "activity_code": row_attrs["activity_code"]})

    if ts_name:
        print(f"Timesheet doc_name existed {ts_name}")
        
        # existed
        payload = {}
        payload["doctype"] = "Timesheet"
        payload["name"] = ts_name["name"]
        payload["employee"] = row_attrs["employee_code"]
        payload["customer"] = company
        payload["parent_project"] = row_attrs["project_code"]
        payload["custom_task"] = row_attrs["task"]
        payload["custom_phase"] = row_attrs["phase_name"]
        payload["custom_activity"] = row_attrs["activity_code"]
        payload["custom_parent_task"] = parent_task_doc['subject']
        payload["custom_parent_related"] = parent_related
        payload["custom_timelog"] = ts_timelog
        # payload["start_date"] = row_attrs["phase_name"]
        time_logs = []
        for date, hrs in time_logs_date.items():
            # check task here for update or new
            task_doc = ts.get_task_doc_by_fields({"subject": row_attrs['subject'], 
                                                  "project_code": row_attrs['project_code'], 
                                                  "phase_name": row_attrs['phase_name'], 
                                                  "activity_code": row_attrs['activity_code'],
                                                  "is_group": 0, 
                                                  "parent_task": parent_task_doc['name']})
            
            if task_doc is None:
                print("task is not exists")
                # new task for timelogs
                task_payload = {
                    "subject": row_attrs["subject"],
                    "project_code": row_attrs["project_code"],
                    "phase_name": row_attrs["phase_name"],
                    "activity_code": row_attrs['activity_code'],
                    "is_group": 0,
                    "progress": row_attrs["progress"],
                    "task_status": row_attrs["task_status"],
                    "parent_task": parent_task_doc['name']
                }
                task_doc = ts.create_task(task_payload)
            else:
                # update task status
                task_payload = {
                    "phase_name": row_attrs["phase_name"],
                    "progress": row_attrs["progress"],
                    "task_status": row_attrs["task_status"],
                }
                ts.update_task(task_doc, task_payload)
                # print(f"new task doc: {task_doc}")

            if task_doc is None:
                print("fixme")
                continue

            hrs_ = 0.0
            try:
                hrs_ = float(hrs)
            except ValueError:
                hrs_ = 0.0
                
            time_logs.append({
                    "activity_type": row_attrs['activity_code'],
                    "from_time": date.strftime("%Y-%m-%d"),
                    "hours": hrs_,
                    "project": row_attrs['project_code'],
                    "task": task_doc['name'],
                    # "completed": excel_task_status == "Done",
                })
        payload["time_logs"] = time_logs
        # payload["is_group"] = 1
        print(f"{payload}")
        conn.update(payload)
        # pass
    else:
        # create new
        print(f"Timesheet need create new")
        payload = {}
        payload["doctype"] = "Timesheet"
        payload["employee"] = row_attrs["employee_code"]
        payload["parent_project"] = row_attrs["project_code"]
        payload["customer"] = company
        payload["custom_task"] = row_attrs["task"]
        payload["custom_phase"] = row_attrs["phase_name"]
        payload["custom_activity"] = row_attrs["activity_code"]
        payload["custom_parent_task"] = parent_task_doc['subject']
        payload["custom_parent_related"] = parent_related
        payload["custom_timelog"] = ts_timelog
        # payload["start_date"] = row_attrs["phase_name"]

        time_logs = []
        for date, hrs in time_logs_date.items():
            # check task here for update or new
            task_doc = ts.get_task_doc_by_fields({
                                "subject": row_attrs['subject'], 
                                "project_code": row_attrs['project_code'], 
                                "phase_name": row_attrs['phase_name'], 
                                "activity_code": row_attrs['activity_code'],
                                "is_group": 0, 
                                "parent_task": parent_task_doc['name']})
            if task_doc is None:
                print("task is not exists")
                # new task for timelogs
                task_payload = {
                    "subject": row_attrs["subject"],
                    "project_code": row_attrs["project_code"],
                    "phase_name": row_attrs["phase_name"],
                    "activity_code": row_attrs['activity_code'],
                    "is_group": 0,
                    "progress": row_attrs["progress"],
                    "task_status": row_attrs["task_status"],
                    "parent_task": parent_task_doc['name']
                }
                task_doc = ts.create_task(task_payload)
                # print(f"new task doc: {task_doc}")
            else:
                # update task status
                task_payload = {
                    "phase_name": row_attrs["phase_name"],
                    "progress": row_attrs["progress"],
                    "task_status": row_attrs["task_status"],
                }
                ts.update_task(task_doc, task_payload)

            if task_doc is None:
                print("fixme")
                continue

            hrs_ = 0.0
            try:
                hrs_ = float(hrs)
            except ValueError:
                hrs_ = 0.0
            
            time_logs.append({
                    "activity_type": row_attrs['activity_code'],
                    "from_time": date.strftime("%Y-%m-%d"),
                    "hours": hrs_,
                    "project": row_attrs['project_code'],
                    "task": task_doc['name'],
                    # "completed": excel_task_status == "Done",
                })
        payload["time_logs"] = time_logs
        # payload["is_group"] = 1
        print(f"{payload}")
        ts.create_timesheet(payload)

    return True

In [11]:
# if row is empty
    # skip this row
# if col_B == "P"
    # handle as parent task
    # if col_A is not empty (parent task is created)
        # handle update parent task if any changes (compare hash)
    # else
        # handle as new parent task
# else
    # handle as task with timesheet
    # if col_A is not empty (timesheet is created)
        # handle update timesheet if any changes (compare hash)
    # else
        # handle as new timesheet

In [12]:
no_project_code_list = []
no_employee_code_list = []

In [13]:
def handle_timesheet_row(idx, row_data, row_date_header, company):
    global current_parent_task_row
    global current_parent_task_doc
    row_attrs = mapping_row_with_attr(row_data)
    # NO project code
    if is_row_empty(row_attrs):
        return False

    # project code is not found ?
    print(f"Project code: {row_attrs['project_code']}")
    project_doc = ts.get_project_doc(row_attrs['project_code'])
    if project_doc is None:
        print(f"Project is not found: {row_attrs['project_code']}")
        no_project_code_list.append({
            "idx": idx,
            "project_code": row_attrs['project_code']})
        return False
    
    if row_attrs['row_type'] == "P":
        ret = handle_timesheet_row_with_parent_task(idx, row_data, row_attrs, row_date_header, company)
        current_parent_task_row = idx
        current_parent_task_doc = ret
        return True
    else:
        # Handle with employee_name column: employee name is not found?
        if row_attrs['employee_name'] == "":
            return False
        print(f"Employee name: {row_attrs['employee_name']}")
        employee_doc = ts.get_employee_doc_by_name(row_attrs['employee_name'])
        if employee_doc is None:
            print(f"Employee is not found: {row_attrs['employee_name']}")
            no_employee_code_list.append({
                "idx": idx,
                "employee_name": row_attrs['employee_name']})
            return False
        else:
            print(f"Employee code: {employee_doc['name']}")
            row_attrs['employee_code'] = employee_doc['name']

        # handle as task with timesheet
        ret = handle_timesheet_row_as_task(idx, row_data, row_attrs, row_date_header, 
                                            current_parent_task_row, current_parent_task_doc, company)
        return ret
        

In [14]:
def handle_timesheet_sheet():
    try:
        # row_date_header = await get_row_date_header()
        # all_records = await ggSheet.get_all_row_values()
        
        current_parent_task_row = 0
        for idx, row_data in enumerate(all_records):
            if idx >= row_start and idx <= row_end:
                print(f"----- row idx: {idx} -----")
                # print(row_data)
                handle_timesheet_row(idx, row_data, row_date_header, company)
                # break
    except Exception as err:
        print(f"Handle timesheet {worksheet_name} failed with: {err}")
        print(traceback.format_exc())

In [16]:
# Team 1:
# https://docs.google.com/spreadsheets/d/1M9T5wt7SIF6-nytkgDEK-LoSGfFgWxlExAy1qo6EPhU/edit?usp=sharing
 
# Team 2:
# https://docs.google.com/spreadsheets/d/1FFkyht1r6aWqgh4i7EX6AkZRxCuSWen8qs75DaTCH_4/edit#gid=1235922659
 
# Team 3:
# https://docs.google.com/spreadsheets/d/1hO5er83L8qU6UITbsjacZg_A6SPW8MuFRBEfwQd1dys/edit?usp=sharing
 
# Team 4:
# https://docs.google.com/spreadsheets/d/1p1vffmF2H-rOSdrNjdqMAXYTFlf9HkBX7FJjk2l4OwA/edit?usp=sharing
 
# Team 5:
# https://docs.google.com/spreadsheets/d/1vTy7Qout9WoeZa31lHLQUw6CZecjEJOKUQlDIAmk1vk/edit?usp=drivesdk
 
# Team 6:
# https://docs.google.com/spreadsheets/d/14m-Sk89NxBeh9pnqrtYr-Vdi92INzIFF1uxO7lpTDuk/edit?usp=sharing
 
# Team RnD:
# https://docs.google.com/spreadsheets/d/15JPQLVqA_WDLmK00AsCnYrv1xqGzWwV9xbung_JDBmU/edit?usp=sharing
 
# Team Civil:
# https://docs.google.com/spreadsheets/d/1qiVQptv2NK08Nr44ZapI8QTEHoDf1DhSpN0jkEe9_Ks/edit?usp=sharing
 
# Team BAP:
# https://docs.google.com/spreadsheets/d/11y58PE9RM16JozFR8yT-oV3K4etIsp2oFmiCsYVaydk/edit#gid=1885142845
 
# ISO Team:
# https://docs.google.com/spreadsheets/d/1KCN11hdITY1R0LpiGmgcGshOCq1U4J0r/edit#gid=646750382

### BAP

In [19]:
url_file="https://docs.google.com/spreadsheets/d/11y58PE9RM16JozFR8yT-oV3K4etIsp2oFmiCsYVaydk/edit#gid=1885142845"
worksheet_name="BAP"

In [20]:
ggSheet = GGSheet(url_file, worksheet_name)

In [21]:
ggSheet = GGSheet(url_file, worksheet_name)
row_date_header = await ggSheet.get_row_date_header(row_of_date)
all_records = await ggSheet.get_all_row_values()

In [22]:
company     = "BAP"
no_project_code_list = []
no_employee_code_list = []
handle_timesheet_sheet()

----- row idx: 5 -----
Project code: HR01_ACO.00
----- row idx: 6 -----
Project code: HR01_ACO.00
Employee name: Nhật Phạm Hoàng
employee: {'name': 'HR-EMP-00007'}
Employee code: HR-EMP-00007
{datetime.datetime(2024, 4, 27, 0, 0): '8', datetime.datetime(2024, 4, 29, 0, 0): '8', datetime.datetime(2024, 4, 30, 0, 0): '8', datetime.datetime(2024, 5, 1, 0, 0): '8'}
['Apr-27-2024', 'Apr-29-2024', 'Apr-30-2024', 'May-01-2024']
Timesheet doc_name existed {'name': 'TS-27-05-24-10016_HR-EMP-00007'}
{'doctype': 'Timesheet', 'name': 'TS-27-05-24-10016_HR-EMP-00007', 'employee': 'HR-EMP-00007', 'customer': 'BAP', 'parent_project': 'HR01_ACO.00', 'custom_task': 'Nghỉ lễ', 'custom_phase': 'Other', 'custom_activity': '2000', 'custom_parent_task': 'Other', 'custom_parent_related': 1, 'custom_timelog': 'Apr-27-2024', 'time_logs': [{'activity_type': '2000', 'from_time': '2024-04-27', 'hours': 8.0, 'project': 'HR01_ACO.00', 'task': 'o2i6q9ak2h'}, {'activity_type': '2000', 'from_time': '2024-04-29', 'hour

In [21]:
print(f"Not found project code list: {no_project_code_list}")
print(f"Not found employee name list: {no_employee_code_list}")


Not found project code list: []
Not found employee name list: []


### TEAM1

In [22]:
url_file = "https://docs.google.com/spreadsheets/d/1M9T5wt7SIF6-nytkgDEK-LoSGfFgWxlExAy1qo6EPhU/edit#gid=1885142845"
# url_file = "https://docs.google.com/spreadsheets/d/1M9T5wt7SIF6-nytkgDEK-LoSGfFgWxlExAy1qo6EPhU/edit?usp=sharing
worksheet_name = "T1"
company = "ACO"

In [23]:
await ggSheet.set_worksheet(url_file, worksheet_name)

In [24]:
row_date_header = await ggSheet.get_row_date_header(row_of_date)
all_records = await ggSheet.get_all_row_values()

In [25]:
no_project_code_list = []
no_employee_code_list = []
handle_timesheet_sheet()

----- row idx: 5 -----
Project code: P16217_ANV.00
{'subject': 'Other', 'project_code': 'P16217_ANV.00', 'phase_name': 'Construction', 'activity_code': '0000', 'progress': '', 'is_group': 1, 'task_status': 'Open'}
----- row idx: 6 -----
Project code: P16217_ANV.00
Employee name: Hưng Trần Ngọc
employee: {'name': 'HR-EMP-00042'}
Employee code: HR-EMP-00042
{datetime.datetime(2024, 4, 4, 0, 0): '4', datetime.datetime(2024, 4, 11, 0, 0): '4'}
['Apr-04-2024', 'Apr-11-2024']
Timesheet need create new
task is not exists
{'doctype': 'Timesheet', 'employee': 'HR-EMP-00042', 'parent_project': 'P16217_ANV.00', 'customer': 'ACO', 'custom_task': 'Họp dự án', 'custom_phase': 'Construction', 'custom_activity': '2001', 'custom_parent_task': 'Other', 'custom_parent_related': 1, 'custom_timelog': 'Apr-04-2024', 'time_logs': [{'activity_type': '2001', 'from_time': '2024-04-04', 'hours': 4.0, 'project': 'P16217_ANV.00', 'task': 'o7afqpnn40'}, {'activity_type': '2001', 'from_time': '2024-04-11', 'hours': 

In [26]:
print(f"Not found project code list: {no_project_code_list}")
print(f"Not found employee name list: {no_employee_code_list}")

Not found project code list: []
Not found employee name list: []


### TEAM_2

In [27]:
url_file="https://docs.google.com/spreadsheets/d/1FFkyht1r6aWqgh4i7EX6AkZRxCuSWen8qs75DaTCH_4/edit#gid=996503932"
# url_file="https://docs.google.com/spreadsheets/d/1FFkyht1r6aWqgh4i7EX6AkZRxCuSWen8qs75DaTCH_4/edit#gid=1235922659
# copied
# url_file = "https://docs.google.com/spreadsheets/d/1rgen0jYUoAzHRelVi6c2ATJs3IzYPD20wMcT63inKoI/edit#gid=996503932"
worksheet_name="T2"
company="ACO"

In [28]:
await ggSheet.set_worksheet(url_file, worksheet_name)

In [29]:
row_date_header = await ggSheet.get_row_date_header(row_of_date)
all_records = await ggSheet.get_all_row_values()

In [30]:
# idx = 165
# row_data = all_records[idx]
            
# print(f"----- row idx: {idx} -----")
# print(row_data)
# time_logs_date, time_logsdate_string = mapping_row_with_time_log(row_data, row_date_header)
# print(time_logs_date)
# print(time_logsdate_string)

In [31]:
no_project_code_list = []
no_employee_code_list = []
handle_timesheet_sheet()

----- row idx: 5 -----
Project code: HR01_ACO.00
----- row idx: 6 -----
Project code: HR01_ACO.00
Employee name: Bình Trịnh Thanh
employee: {'name': 'HR-EMP-00020'}
Employee code: HR-EMP-00020
{datetime.datetime(2024, 4, 18, 0, 0): '8', datetime.datetime(2024, 4, 27, 0, 0): '4'}
['Apr-18-2024', 'Apr-27-2024']
Timesheet need create new
task is not exists
{'doctype': 'Timesheet', 'employee': 'HR-EMP-00020', 'parent_project': 'HR01_ACO.00', 'customer': 'ACO', 'custom_task': '', 'custom_phase': 'Other', 'custom_activity': '2000', 'custom_parent_task': 'Other', 'custom_parent_related': 1, 'custom_timelog': 'Apr-18-2024', 'time_logs': [{'activity_type': '2000', 'from_time': '2024-04-18', 'hours': 8.0, 'project': 'HR01_ACO.00', 'task': 'o8lrj8nauv'}, {'activity_type': '2000', 'from_time': '2024-04-27', 'hours': 4.0, 'project': 'HR01_ACO.00', 'task': 'o8lrj8nauv'}]}
----- row idx: 7 -----
Project code: HR01_ACO.00
Employee name: Sokkheang Chan
employee: {'name': 'HR-EMP-00050'}
Employee code: 

In [32]:
print(f"Not found project code list: {no_project_code_list}")
print(f"Not found employee name list: {no_employee_code_list}")

Not found project code list: [{'idx': 643, 'project_code': 'NC02024_CBA.00'}, {'idx': 644, 'project_code': 'NC02024_CBA.00'}, {'idx': 645, 'project_code': 'NC02024_CBA.00'}, {'idx': 646, 'project_code': 'NC02024_CBA.00'}, {'idx': 647, 'project_code': 'NC02024_CBA.00'}, {'idx': 648, 'project_code': 'NC02024_CBA.00'}]
Not found employee name list: []


### TEAM3

In [33]:
# team_3
url_file="https://docs.google.com/spreadsheets/d/1hO5er83L8qU6UITbsjacZg_A6SPW8MuFRBEfwQd1dys/edit#gid=370020233"
# url_file="https://docs.google.com/spreadsheets/d/1hO5er83L8qU6UITbsjacZg_A6SPW8MuFRBEfwQd1dys/edit?usp=sharing
worksheet_name="T3"
company="ACO"

In [34]:
await ggSheet.set_worksheet(url_file, worksheet_name)

In [35]:
row_date_header = await ggSheet.get_row_date_header(row_of_date)
all_records = await ggSheet.get_all_row_values()

In [36]:
no_project_code_list = []
no_employee_code_list = []
handle_timesheet_sheet()

----- row idx: 5 -----
Project code: P000_ACO.00
{'subject': 'Other', 'project_code': 'P000_ACO.00', 'phase_name': 'Other', 'activity_code': '0000', 'progress': '', 'is_group': 1, 'task_status': 'Open'}
----- row idx: 6 -----
Project code: P000_ACO.00
Employee name: Trung Đỗ Thành
employee: {'name': 'HR-EMP-00041'}
Employee code: HR-EMP-00041
{datetime.datetime(2024, 5, 6, 0, 0): '2', datetime.datetime(2024, 5, 13, 0, 0): '2', datetime.datetime(2024, 5, 20, 0, 0): '2'}
['May-06-2024', 'May-13-2024', 'May-20-2024']
Timesheet need create new
task is not exists
{'doctype': 'Timesheet', 'employee': 'HR-EMP-00041', 'parent_project': 'P000_ACO.00', 'customer': 'ACO', 'custom_task': 'Họp nội bộ', 'custom_phase': 'Other', 'custom_activity': '2000', 'custom_parent_task': 'Other', 'custom_parent_related': 1, 'custom_timelog': 'May-06-2024', 'time_logs': [{'activity_type': '2000', 'from_time': '2024-05-06', 'hours': 2.0, 'project': 'P000_ACO.00', 'task': 'qkc57ju6go'}, {'activity_type': '2000', '

In [37]:
print(f"Not found project code list: {no_project_code_list}")
print(f"Not found employee name list: {no_employee_code_list}")

Not found project code list: []
Not found employee name list: []


### TEAM 4

In [38]:
# team_4
url_file="https://docs.google.com/spreadsheets/d/1p1vffmF2H-rOSdrNjdqMAXYTFlf9HkBX7FJjk2l4OwA/edit#gid=1885142845"
# url_file="https://docs.google.com/spreadsheets/d/1p1vffmF2H-rOSdrNjdqMAXYTFlf9HkBX7FJjk2l4OwA/edit?usp=sharing
worksheet_name="T4"
company="ACO"

In [39]:
await ggSheet.set_worksheet(url_file, worksheet_name)

In [40]:
row_date_header = await ggSheet.get_row_date_header(row_of_date)
all_records = await ggSheet.get_all_row_values()

In [41]:
no_project_code_list = []
no_employee_code_list = []
handle_timesheet_sheet()

----- row idx: 5 -----
Project code: HR01_ACO.00
----- row idx: 6 -----
Project code: HR01_ACO.00
Employee name: An Nguyễn Bình
employee: {'name': 'HR-EMP-00003'}
Employee code: HR-EMP-00003
{datetime.datetime(2024, 4, 18, 0, 0): '8'}
['Apr-18-2024']
Timesheet need create new
task is not exists
{'doctype': 'Timesheet', 'employee': 'HR-EMP-00003', 'parent_project': 'HR01_ACO.00', 'customer': 'ACO', 'custom_task': 'Nghỉ lễ giỗ tổ', 'custom_phase': 'Construction', 'custom_activity': '4001', 'custom_parent_task': 'Other', 'custom_parent_related': 1, 'custom_timelog': 'Apr-18-2024', 'time_logs': [{'activity_type': '4001', 'from_time': '2024-04-18', 'hours': 8.0, 'project': 'HR01_ACO.00', 'task': 'qmil17n2un'}]}
----- row idx: 7 -----
Project code: HR01_ACO.00
Employee name: Son Trần Thị Minh
employee: {'name': 'HR-EMP-00005'}
Employee code: HR-EMP-00005
{datetime.datetime(2024, 4, 18, 0, 0): '8'}
['Apr-18-2024']
Timesheet need create new
{'doctype': 'Timesheet', 'employee': 'HR-EMP-00005', 

In [42]:
print(f"Not found project code list: {no_project_code_list}")
print(f"Not found employee name list: {no_employee_code_list}")

Not found project code list: []
Not found employee name list: []


### TEAM 5

In [43]:
# team_5
url_file="https://docs.google.com/spreadsheets/d/1vTy7Qout9WoeZa31lHLQUw6CZecjEJOKUQlDIAmk1vk/edit#gid=1742134070"
# url_file="https://docs.google.com/spreadsheets/d/1vTy7Qout9WoeZa31lHLQUw6CZecjEJOKUQlDIAmk1vk/edit?usp=drivesdk
worksheet_name="T5"
company="ACO"

In [44]:
await ggSheet.set_worksheet(url_file, worksheet_name)

In [45]:
row_date_header = await ggSheet.get_row_date_header(row_of_date)
all_records = await ggSheet.get_all_row_values()

In [46]:
no_project_code_list = []
no_employee_code_list = []
handle_timesheet_sheet()

----- row idx: 5 -----
Project code: P39423_KNCR.00
{'subject': 'THIẾT KẾ KỸ THUẬT', 'project_code': 'P39423_KNCR.00', 'phase_name': 'Design Development', 'activity_code': '0000', 'progress': '', 'is_group': 1, 'task_status': 'Open'}
----- row idx: 6 -----
Project code: P39423_KNCR.00
----- row idx: 7 -----
Project code: P39423_KNCR.00
Employee name: Thành Hồ Đức Thiện
employee: {'name': 'HR-EMP-00008'}
Employee code: HR-EMP-00008
{datetime.datetime(2024, 4, 1, 0, 0): '8', datetime.datetime(2024, 4, 2, 0, 0): '8', datetime.datetime(2024, 4, 3, 0, 0): '8', datetime.datetime(2024, 4, 4, 0, 0): '8', datetime.datetime(2024, 4, 5, 0, 0): '8', datetime.datetime(2024, 4, 6, 0, 0): '4', datetime.datetime(2024, 4, 8, 0, 0): '8', datetime.datetime(2024, 4, 9, 0, 0): '8', datetime.datetime(2024, 4, 10, 0, 0): '8'}
['Apr-01-2024', 'Apr-02-2024', 'Apr-03-2024', 'Apr-04-2024', 'Apr-05-2024', 'Apr-06-2024', 'Apr-08-2024', 'Apr-09-2024', 'Apr-10-2024']
Timesheet need create new
task is not exists
{'do

In [47]:
print(f"Not found project code list: {no_project_code_list}")
print(f"Not found employee name list: {no_employee_code_list}")

Not found project code list: []
Not found employee name list: []


### TEAM 6

In [48]:
# team_6
url_file="https://docs.google.com/spreadsheets/d/14m-Sk89NxBeh9pnqrtYr-Vdi92INzIFF1uxO7lpTDuk/edit#gid=1175515436"
# url_file="https://docs.google.com/spreadsheets/d/14m-Sk89NxBeh9pnqrtYr-Vdi92INzIFF1uxO7lpTDuk/edit?usp=sharing
worksheet_name="T6"
company="ACO"

In [49]:
await ggSheet.set_worksheet(url_file, worksheet_name)
row_date_header = await ggSheet.get_row_date_header(row_of_date)
all_records = await ggSheet.get_all_row_values()

In [50]:
no_project_code_list = []
no_employee_code_list = []
handle_timesheet_sheet()

----- row idx: 5 -----
Project code: HR01_ACO.00
----- row idx: 6 -----
Project code: HR01_ACO.00
Employee name: Thiện Lê Phú
employee: {'name': 'HR-EMP-00043'}
Employee code: HR-EMP-00043
{datetime.datetime(2024, 4, 18, 0, 0): '8', datetime.datetime(2024, 4, 27, 0, 0): '4', datetime.datetime(2024, 4, 29, 0, 0): '8', datetime.datetime(2024, 4, 30, 0, 0): '8', datetime.datetime(2024, 5, 1, 0, 0): '8'}
['Apr-18-2024', 'Apr-27-2024', 'Apr-29-2024', 'Apr-30-2024', 'May-01-2024']
Timesheet need create new
task is not exists
{'doctype': 'Timesheet', 'employee': 'HR-EMP-00043', 'parent_project': 'HR01_ACO.00', 'customer': 'ACO', 'custom_task': '', 'custom_phase': 'Other', 'custom_activity': '4001', 'custom_parent_task': 'Other', 'custom_parent_related': 1, 'custom_timelog': 'Apr-18-2024', 'time_logs': [{'activity_type': '4001', 'from_time': '2024-04-18', 'hours': 8.0, 'project': 'HR01_ACO.00', 'task': 'qp3m1lghui'}, {'activity_type': '4001', 'from_time': '2024-04-27', 'hours': 4.0, 'project':

In [51]:
print(f"Not found project code list: {no_project_code_list}")
print(f"Not found employee name list: {no_employee_code_list}")

Not found project code list: [{'idx': 103, 'project_code': 'P000'}, {'idx': 104, 'project_code': 'P000'}, {'idx': 105, 'project_code': 'P000'}, {'idx': 106, 'project_code': 'P000'}, {'idx': 157, 'project_code': 'P40424_[TBD].00'}, {'idx': 158, 'project_code': 'P40424_[TBD].00'}, {'idx': 159, 'project_code': 'P40424_[TBD].00'}, {'idx': 160, 'project_code': 'P40424_[TBD].00'}, {'idx': 161, 'project_code': 'P40424_[TBD].00'}]
Not found employee name list: []


### TEAM CIVIL

In [52]:
# team_civil
url_file = "https://docs.google.com/spreadsheets/d/1qiVQptv2NK08Nr44ZapI8QTEHoDf1DhSpN0jkEe9_Ks/edit#gid=26030579"
# url_file = "https://docs.google.com/spreadsheets/d/1qiVQptv2NK08Nr44ZapI8QTEHoDf1DhSpN0jkEe9_Ks/edit?usp=sharing
worksheet_name="CIVIL"
company="ACO"

In [53]:
await ggSheet.set_worksheet(url_file, worksheet_name)
row_date_header = await ggSheet.get_row_date_header(row_of_date)
all_records = await ggSheet.get_all_row_values()

In [54]:
no_project_code_list = []
no_employee_code_list = []
handle_timesheet_sheet()

----- row idx: 5 -----
----- row idx: 6 -----
Project code: P33422_HTP.00
----- row idx: 7 -----
Project code: P33422_HTP.00
----- row idx: 8 -----
Project code: P33422_HTP.00
Employee name: Minh Nguyễn Thành
employee: {'name': 'HR-EMP-00078'}
Employee code: HR-EMP-00078
{}
[]
Timesheet need create new
{'doctype': 'Timesheet', 'employee': 'HR-EMP-00078', 'parent_project': 'P33422_HTP.00', 'customer': 'ACO', 'custom_task': 'MB tổ chức giao thông & chi tiết', 'custom_phase': 'Other', 'custom_activity': '2303', 'custom_parent_task': 'THIẾT KẾ THI CÔNG', 'custom_parent_related': 2, 'custom_timelog': '', 'time_logs': []}
----- row idx: 9 -----
Project code: P33422_HTP.00
Employee name: Minh Nguyễn Thành
employee: {'name': 'HR-EMP-00078'}
Employee code: HR-EMP-00078
{}
[]
Timesheet need create new
{'doctype': 'Timesheet', 'employee': 'HR-EMP-00078', 'parent_project': 'P33422_HTP.00', 'customer': 'ACO', 'custom_task': 'Đấu nối đường Ven Biển hiện trạng', 'custom_phase': 'Other', 'custom_activ

In [55]:
print(f"Not found project code list: {no_project_code_list}")
print(f"Not found employee name list: {no_employee_code_list}")

Not found project code list: [{'idx': 567, 'project_code': 'INTERSHIP'}]
Not found employee name list: []


### TEAM RND

In [56]:
# team_rnd
url_file="https://docs.google.com/spreadsheets/d/15JPQLVqA_WDLmK00AsCnYrv1xqGzWwV9xbung_JDBmU/edit#gid=1885142845"
# url_file="https://docs.google.com/spreadsheets/d/15JPQLVqA_WDLmK00AsCnYrv1xqGzWwV9xbung_JDBmU/edit?usp=sharing
worksheet_name="RnD"
company="ACO"

In [57]:
await ggSheet.set_worksheet(url_file, worksheet_name)
row_date_header = await ggSheet.get_row_date_header(row_of_date)
all_records = await ggSheet.get_all_row_values()

In [58]:
no_project_code_list = []
no_employee_code_list = []
handle_timesheet_sheet()

----- row idx: 5 -----
Project code: P000_ACO.00
----- row idx: 6 -----
Project code: P000_ACO.00
Employee name: Việt Nguyễn Hồng
employee: {'name': 'HR-EMP-00040'}
Employee code: HR-EMP-00040
{datetime.datetime(2024, 5, 20, 0, 0): '2'}
['May-20-2024']
Timesheet need create new
task is not exists
{'doctype': 'Timesheet', 'employee': 'HR-EMP-00040', 'parent_project': 'P000_ACO.00', 'customer': 'ACO', 'custom_task': 'Gernal Task', 'custom_phase': 'Other', 'custom_activity': '2000', 'custom_parent_task': 'Other', 'custom_parent_related': 1, 'custom_timelog': 'May-20-2024', 'time_logs': [{'activity_type': '2000', 'from_time': '2024-05-20', 'hours': 2.0, 'project': 'P000_ACO.00', 'task': 'qsf4q9s064'}]}
----- row idx: 7 -----
----- row idx: 8 -----
----- row idx: 9 -----
----- row idx: 10 -----
----- row idx: 11 -----
----- row idx: 12 -----
Project code: I00124_RSCI.00
{'subject': 'Other', 'project_code': 'I00124_RSCI.00', 'phase_name': 'Other', 'activity_code': '0000', 'progress': '', 'is

In [40]:
print(f"Not found project code list: {no_project_code_list}")
print(f"Not found employee name list: {no_employee_code_list}")

Not found project code list: []
Not found employee name list: []
